## Setup Models

In [9]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import ChatOllama

#Setup the LLM
model = ChatOllama(model="llama3.2")
# model = ChatOpenAI(model="gpt-5")

#Setup the Embedding
embedding = OpenAIEmbeddings(model="text-embedding-3-small")


## 03.02. Add  Product Pricing function tool

In [10]:
import pandas as pd
from langchain_core.tools import tool

#Load the laptop product pricing CSV into a Pandas dataframe.
product_pricing_df = pd.read_csv("data/Laptop pricing.csv")
print(product_pricing_df)

@tool
def get_laptop_price(laptop_name:str) -> int :
    """
    This function returns the price of a laptop, given its name as input.
    It performs a substring match between the input name and the laptop name.
    If a match is found, it returns the price of the laptop.
    If there is NO match found, it returns -1
    """

    #Filter Dataframe for matching names
    match_records_df = product_pricing_df[
                        product_pricing_df["Name"].str.contains(
                                                "^" + laptop_name, case=False)
                        ]
    #Check if a record was found, if not return -1
    if len(match_records_df) == 0 :
        return -1
    else:
        return match_records_df["Price"].iloc[0]

#Test the tool. Before running the test, comment the @tool annotation
# print(get_laptop_price("alpha"))
# print(get_laptop_price("testing"))


            Name  Price  ShippingDays
0  AlphaBook Pro   1499             2
1     GammaAir X   1399             7
2  SpectraBook S   2499             7
3   OmegaPro G17   2199            14
4  NanoEdge Flex   1699             2


## 03.03. Add Product Features Retrieval Tool

In [11]:
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the product featuers document.
loader=PyPDFLoader("./data/Laptop product descriptions.pdf")
docs = loader.load()

print(docs[0])

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

#Create a vector store with Chroma
prod_feature_store = Chroma.from_documents(
    documents=splits,
    embedding=embedding
)

get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about Laptops. It lists the available laptops
    and their features including CPU, memory, storage, design and advantages
    """
)

#Test the product feature store
print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

page_content='Fictional Laptop Descriptions
AlphaBook Pro
The AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,
and a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of
power and portability.
GammaAir X
GammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB
NVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a
portable design.
SpectraBook S
Designed for power users, SpectraBook S features an Intel Core i9 processor, 64GB RAM, and a
massive 2TB SSD. This workstation-class laptop is perfect for intensive tasks like video editing and
3D rendering.
OmegaPro G17
OmegaPro G17 is a gaming powerhouse with a Ryzen 9 5900HX CPU, 32GB RAM, and a 1TB
SSD. Designed for gamers, it features a 17-inch display with a high refresh rate and powerful
graphics card for the best gaming experience.
NanoEdge Flex
NanoEdge Flex is a versatile 2-in-1 laptop with 

## 03.04.Setup a Product QnA chatbot

In [12]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create a System prompt to provide a persona to the chatbot
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

#Create a list of tools available
tools = [get_laptop_price, get_product_features]

#Create memory across questions in a conversation (conversation memory)
checkpointer=MemorySaver()

#Create a Product QnA Agent. This is actual a graph in langGraph
product_QnA_agent=create_react_agent(
                                model=model, #LLM to use
                                tools=tools, #List of tools to use
                                prompt=system_prompt, #The system prompt
                                debug=False, #Debugging turned on if needed
                                checkpointer=checkpointer #For conversation memory
)


In [13]:
#Setup chatbot
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the features and pricing for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (3c0a469c-ea70-4fc7-8aad-acbe32ecddaf)
 Call ID: 3c0a469c-ea70-4fc7-8aad-acbe32ecddaf
  Args:
    query: GammaAir
================================= Tool Message =================================
Name: Get_Product_Features

Fictional Laptop Descriptions
AlphaBook Pro
The AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,
and a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of
power and portability.
GammaAir X
GammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB
NVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a
portable design.
SpectraBook S
Designed for power users, SpectraBook S features an

## 03.05. Execute the Product QnA Chatbot

In [14]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")


----------------------------------------
USER : Hello
AGENT : Hello! How can I assist you today? Are you looking to purchase one of our laptops or have questions about their features?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : We have several laptops to choose from. Our top-of-the-line options include:

* The AlphaBook Pro, perfect for professionals on the go with its 12th Gen Intel i7 processor and 16GB of DDR4 RAM.
* The SpectraBook S, designed for power users with its Intel Core i9 processor and massive 2TB SSD.
* The OmegaPro G17, a gaming powerhouse with a Ryzen 9 5900HX CPU and powerful graphics card.

If you're looking for something more portable, our GammaAir X is a great option with an AMD Ryzen 7 processor and 32GB of DDR4 memory.

What type of use do you plan to put your laptop to?
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Here is the list of available laptops:

1. AlphaBook Pro


In [15]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  SpectraBook")
execute_prompt("USER 2", config_2, "Tell me about the features of  GammaAir")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")




USER 1: The SpectraBook is a versatile ultrabook that offers a range of features to suit different user needs.

Key specifications include:

* Processor: Up to 12th Gen Intel Core i7
* Memory: Up to 16GB DDR4 RAM
* Storage: Up to 1TB SSD
* Display: High-resolution display with various size options (11.6", 13.3", and 15.6 inches)
* Battery Life: Up to 14 hours of battery life

The SpectraBook is designed for productivity, portability, and versatility. Its lightweight design and powerful processor make it ideal for professionals on the go, students, and anyone who needs a reliable laptop for everyday tasks.

USER 2: The GammaAir features:

* AMD Ryzen 7 processor
* 32GB of DDR4 memory
* 512GB NVMe SSD

USER 1: The SpectraBook is priced at $2,499. This price may vary depending on the specific configuration and location. The price includes:

* Base model with 12th Gen Intel Core i5 processor, 8GB RAM, and 256GB SSD
* Optional upgrades such as 16GB RAM, 512GB SSD, and Intel Core i7 process